# Retraining StyleGAN2 on Kaggle

Inspired by Jeff Heaton's Google Colab and Docker examples on training StyleGAN2

https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_07_3_style_gan.ipynb
https://hub.docker.com/r/heatonresearch/stylegan2-ada


## Background

StyleGAN2 requires a Windows or Linux machine with at least 1 GPU. My computer meets neither requirement so I ran a notebook using Colab's GPU time.

# Clone the StyleGAN2 Repository and Install Packages

Run the code block below to clone the repository and install packages. This is not specific to Kaggle.

In [ ]:
#@title Mount Google Drive (Optional)
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
!git clone https://github.com/NVlabs/stylegan2.git
!pip install ninja tensorflow==1.14

fatal: destination path 'stylegan2' already exists and is not an empty directory.


In [ ]:
!mkdir /content/diamonds/
%cd /content/diamonds/
# website to create link https://sites.google.com/site/gdocs2direct/
!gdown --id 1uKjQDVpbquOb50PxWgFM11kp4O9zDept
%cd /content/

/content/diamonds
Downloading...
From: https://drive.google.com/uc?id=1uKjQDVpbquOb50PxWgFM11kp4O9zDept
To: /content/diamonds/square.zip
100% 202M/202M [00:11<00:00, 17.0MB/s]
/content


If you are running in Kaggle, you need to run the code block below, otherwise during training you will encounter a error like "GLIBCXX_3.4.26 not found error".

If you are not running in Kaggle, hopefully you won't need to run the code block below.

https://www.kaggle.com/valentinstefan/yolov5-person-class

In [ ]:
# you may not need to run this if you are not in Kaggle
!add-apt-repository ppa:ubuntu-toolchain-r/test -y
!apt-get update
!apt-get upgrade libstdc++6 -y

Verify TF version 1.14

In [ ]:
import tensorflow as tf
print(tf.__version__)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

1.14.0


/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

# Convert your images to data format

You need to convert your images to a data format for StyleGAN2. Your images must be square and have dimensions with a power of 2 (ex 256 x 256).

In the command below, you should specify the source path to all your input JPEG images (ie as a zip file) as well as the destination path to store the output dataset.

If you are not in Kaggle, you should change the argument paths.

In [ ]:
!python /content/stylegan2/dataset_tool.py create_from_images /content/dataset/ /content/drive/MyDrive/ML/DiamondGAN/square/
!python /content/stylegan2/dataset_tool.py display /content/dataset/

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
#!apt search libgtk2.0-dev
#!ls -a ~/.local/lib/
!pip install opencv-python
#!pip install libgtk2.0-dev

# Train StyleGAN2 on your images!

If you are not in Kaggle, you should change the argument paths.

In [ ]:
!python /kaggle/working/stylegan2/run_training.py --config=config-f --num-gpus=1 --data-dir=/kaggle/working/dataset --dataset=diamonds --mirror-augment=true --total-kimg=1000 --result-dir=/kaggle/working/results